In [ ]:
import numpy as np
from sklearn.svm import SVC, LinearSVC
import glob
import cv2 as cv
import pdb
from copy import deepcopy
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import os

In [ ]:
def get_positive_descriptors_resnet():
    images_path = os.path.join('data/exemplePozitive', '*.jpg')
    files = glob.glob(images_path)
    num_images = len(files)
    positive_descriptors = []
    model = models.resnet18(pretrained=True)
    layer = model._modules.get('avgpool')
    model.eval()
    print('Calculam descriptorii pt %d imagini pozitive...' % num_images)
    scaler = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    for i in range(num_images):
        print('Procesam exemplul pozitiv numarul %d...' % i)
        img = cv.imread(files[i])
        img = Image.fromarray(img)
        t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
        my_embedding = torch.zeros(512)
        def copy_data(m, i, o):
            my_embedding.copy_(o.data.reshape(o.data.size(1)))
        h = layer.register_forward_hook(copy_data)
        model(t_img)
        h.remove()
        my_embedding = my_embedding.numpy()
        positive_descriptors.append(my_embedding)
    positive_descriptors = np.array(positive_descriptors)
    return positive_descriptors

In [ ]:
def get_negative_descriptors_resnet():
    images_path = os.path.join('data/exempleNegative', '*.jpg')
    files = glob.glob(images_path)
    num_images = len(files)
    negative_descriptors = []
    model = models.resnet18(pretrained=True)
    layer = model._modules.get('avgpool')
    model.eval()
    print('Calculam descriptorii pt %d imagini pozitive...' % num_images)
    scaler = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    for i in range(num_images):
        print('Procesam exemplul pozitiv numarul %d...' % i)
        img = cv.imread(files[i])
        num_rows = img.shape[0]
        num_cols = img.shape[1]
        x = np.random.randint(low=0, high=num_cols - 36, size=35)
        y = np.random.randint(low=0, high=num_rows - 36, size=35)
        for idx in range(len(y)):
                patch = img[y[idx]: y[idx] + 36, x[idx]: x[idx] + 36]
                patch = Image.fromarray(patch)
                t_img = Variable(normalize(to_tensor(scaler(patch))).unsqueeze(0))
                my_embedding = torch.zeros(512)
                def copy_data(m, i, o):
                    my_embedding.copy_(o.data.reshape(o.data.size(1)))
                h = layer.register_forward_hook(copy_data)
                model(t_img)
                h.remove()
                my_embedding = my_embedding.numpy()
                negative_descriptors.append(my_embedding)
    negative_descriptors = np.array(negative_descriptors)
    return negative_descriptors

In [ ]:
positive_features = get_positive_descriptors_resnet()

In [ ]:
negative_features = get_negative_descriptors_resnet()

In [ ]:
print(positive_features.shape)
print(negative_features.shape)

In [ ]:
training_examples = np.concatenate((np.squeeze(positive_features), np.squeeze(negative_features)), axis=0)
train_labels = np.concatenate((np.ones(positive_features.shape[0]), np.zeros(negative_features.shape[0])))

In [ ]:
best_accuracy = 0
best_c = 0
best_model = None
Cs = [10 ** -5, 10 ** -4,  10 ** -3,  10 ** -2, 10 ** -1, 10 ** 0]
for c in Cs:
    print('Antrenam un clasificator pentru c=%f' % c)
    model = LinearSVC(C=c)
    model.fit(training_examples, train_labels)
    acc = model.score(training_examples, train_labels)
    print(acc)
    if acc > best_accuracy:
        best_accuracy = acc
        best_c = c
        best_model = deepcopy(model)

print('Performanta clasificatorului optim pt c = %f' % best_c)

In [ ]:
img_test_negative=os.listdir('dateTest_negative')

In [ ]:
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
for img in img_test_negative:
    img_test=cv.imread('dateTest_negative/'+img)
    img_test= Image.fromarray(img_test)
    t_img = Variable(normalize(to_tensor(scaler(img_test))).unsqueeze(0))
    my_embedding = torch.zeros(512)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = layer.register_forward_hook(copy_data)
    model(t_img)
    h.remove()
    descr = my_embedding.numpy()
    print(best_model.predict([descr]))

In [ ]:
img_test_pozitive=os.listdir('dateTest_pozitive')
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
for img in img_test_pozitive:
    img_test=cv.imread('dateTest_pozitive/'+img)
    img_test= Image.fromarray(img_test)
    t_img = Variable(normalize(to_tensor(scaler(img_test))).unsqueeze(0))
    my_embedding = torch.zeros(512)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = layer.register_forward_hook(copy_data)
    model(t_img)
    h.remove()
    descr = my_embedding.numpy()
    print(best_model.predict([descr]))